# <center> Module 2 Project  - Northwind Database Hypothesis Testing

## <center>Matthew Sparr <br><br> <center><h8>Self-Paced Data Science Program

# <center> Introduction

First we will import all the necessary libraries.

In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from scipy import stats

Next we will connect to the database and grab all the table names.

In [2]:
conn = sql.connect('Northwind_small.sqlite')
cur = conn.cursor()
table_names = list(cur.execute('''SELECT name FROM sqlite_master WHERE TYPE = 'table'
                               ''').fetchall())
table_names

[('Employee',),
 ('Category',),
 ('Customer',),
 ('Shipper',),
 ('Supplier',),
 ('Order',),
 ('Product',),
 ('OrderDetail',),
 ('CustomerCustomerDemo',),
 ('CustomerDemographic',),
 ('Region',),
 ('Territory',),
 ('EmployeeTerritory',)]

Now we will store each table from the database as Pandas Dataframe so that they are easy to work with and analyze. There was a technical error with the 'orders' table, so a CSV file of the table was generated using an outside program.

In [3]:
sql_comm = []
for i in range(0,len(table_names)):
    s = "SELECT * FROM " + table_names[i][0]
    sql_comm.append(s)

employees = pd.read_sql_query(sql_comm[0], conn)
categories = pd.read_sql_query(sql_comm[1], conn)
customers = pd.read_sql_query(sql_comm[2], conn)
shippers = pd.read_sql_query(sql_comm[3], conn)

suppliers = pd.read_sql_query(sql_comm[4], conn)
products = pd.read_sql_query(sql_comm[6], conn)
order_details = pd.read_sql_query(sql_comm[7], conn)
orders = pd.read_csv('Order.csv')

cust_cust_demos = pd.read_sql_query(sql_comm[8], conn)
cust_demos = pd.read_sql_query(sql_comm[9], conn)
regions = pd.read_sql_query(sql_comm[10], conn)
territories = pd.read_sql_query(sql_comm[11], conn)
emp_territories = pd.read_sql_query(sql_comm[12], conn)

And here is a schema of the database.

<img src='Northwind_ERD.png'>

## Q1 -- Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

## <center>H<sub>0: The average quantity of product ordered is the same for orders with and without a discount.

## <center>H<sub>a: The average quantity of product ordered when a discount is given is higher or lower than for orders without a discount.

The above is our first question and both the null and alternative hypotheses. To test them will involve a two-tail test. This is because if the null hypothesis is rejected and there is a correlation between discount and order quantity, we could find that the discount amount either increases or decreases the quantity of product ordered. 

To test our hypotheses we will be using the table 'order_details' and will be looking only at the columns 'Quantity' and 'Discount'. Below we can see that there are no null or missing values in either column. Also, 'Quantity' ranges from 1 to 130 and 'Discount' ranges from 0 to 0.25.

In [4]:
order_details[['Quantity','Discount']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 2 columns):
Quantity    2155 non-null int64
Discount    2155 non-null float64
dtypes: float64(1), int64(1)
memory usage: 33.8 KB


In [5]:
order_details[['Quantity','Discount']].describe()

,Quantity,Discount
count,2155.000000,2155.000000
mean,23.812993,0.056167
std,19.022047,0.083450
min,1.000000,0.000000
25%,10.000000,0.000000
50%,20.000000,0.000000
75%,30.000000,0.100000
max,130.000000,0.250000


Now we will split our order details into two sets: discount and no discount. Running a t-test on the data below then gives us a small p-value of 1.14e-10 which is less than our alpha of 0.05.

In [6]:
no_disc = order_details[order_details['Discount'] == 0].copy()
disc = order_details[order_details['Discount'] > 0].copy()

p = stats.ttest_ind(no_disc.Quantity, disc.Quantity)[1]
p

1.1440924523215966e-10

Since the p-value for our t-test is so low, this allows us to reject the null-hypothesis and accept instead the alternative hypothesis. From this result we can reasonably argue that having a discount does indeed have an impact on the quantity of product ordered. 

For the second part of the question we will need to look at the different discount amounts and see where the size of the discount affects the quantity ordered the most. Let's start by looking at the unique values in 'Discount' below.

In [7]:
order_details.Discount.value_counts()

0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Discount, dtype: int64

Since the count of some discount values are so few compared to others, we will group those discount values together. Below we create 5 buckets of discount values and then add them each to an array so we can iterate through them. We will also be filtering out data where there is no discount given.

In [8]:
disc_5 = order_details['Quantity'][(order_details['Discount'] <= 0.05) & (order_details['Discount'] > 0)]
disc_10 = order_details['Quantity'][(order_details['Discount'] > 0.05) & (order_details['Discount'] <= 0.1)]
disc_15 = order_details['Quantity'][(order_details['Discount'] > 0.1) & (order_details['Discount'] <= 0.15)]
disc_20 = order_details['Quantity'][(order_details['Discount'] > 0.15) & (order_details['Discount'] <= 0.2)]
disc_25 = order_details['Quantity'][order_details['Discount'] > 0.2]
discounts = []
discounts.extend((['0.05 or less', disc_5], ['Between 0.05 and 0.10', disc_10],
                  ['Between 0.10 and 0.15', disc_15], ['Between 0.15 and 0.20', disc_20],
                  ['Greater than 0.20', disc_25]))

We can then view the size of each of these buckets and notice that they are all fairly similar in size.

In [9]:
for i in discounts:
    print(len(i[1]))

192
174
157
161
154


For this second part of the question, we will need a set of  null and alternative hypotheses.

## <center>H<sub>0: The average quantity of product ordered is the same for X and Y amount of discount.

## <center>H<sub>a: The average quantity of product ordered is different for X and Y amount of discount.

In the above hypotheses, X and Y represent two discount levels. Since we have 5 buckets of individual discount amounts, we will start by declaring the lowest one (0.05 or less) X and the next level up (between 0.05 and 0.10) Y and then running our T-test and checking the p-value. We will then repeat this up to the highest discount level giving us 4 seperate hypotheses to test.

In [10]:
for i in range(0, len(discounts)-1):
    X = discounts[i][1]
    y = discounts[i+1][1]
    p = stats.ttest_ind(X, y)[1]
    print(discounts[i][0] + " v.s. " + discounts[i+1][0] + " : " + str(p))

0.05 or less v.s. Between 0.05 and 0.10 : 0.39367155112980046
Between 0.05 and 0.10 v.s. Between 0.10 and 0.15 : 0.15837034442401487
Between 0.10 and 0.15 v.s. Between 0.15 and 0.20 : 0.5433964640207652
Between 0.15 and 0.20 v.s. Greater than 0.20 : 0.5801473824667989


None of the above p-values are below our alpha of 0.05. Therefore we fail to reject the null-hypothesis and cannot conclude that increasing the discount at any level will significantly affect the quantity of product ordered.

### Our findings show that having a discount on a product most likely will increase the quantity of product order but the actual amount of discount is statistically insignificant. 

## Q2 -- Are there certain products that are better to discount than others in order to increase the quantity ordered?

As a follow-up to our initial question, we will investigate whether or not certain categories of items are more advantageous to discount if our goal is to increase total sales.

## <center>H<sub>0: The average quantity of product ordered is the same regardless of category or discount.

## <center>H<sub>a: The average quantity of product ordered varies in correlation with category and discount.

To test our hypothesis we will need data from more than one table so we will need to perform a couple merges. We will first join the 'products' table to the 'order-details' table as 'prod_ord_details'; and then merge that table with the 'categories' table.

In [11]:
prod_ord_details = pd.merge(products, 
                  order_details,
                  left_on='Id',
                  right_on='ProductId',
                  how='left')
prod_cat = pd.merge(prod_ord_details, 
                  categories,
                  left_on='CategoryId',
                  right_on='Id',
                  how='left')
prod_cat.head()

,Id_x,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice_x,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,Id_y,OrderId,ProductId,UnitPrice_y,Quantity,Discount,Id,CategoryName,Description
0,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0,10285/1,10285,1,14.4,45,0.20,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0,10294/1,10294,1,14.4,18,0.00,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
2,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0,10317/1,10317,1,14.4,20,0.00,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
3,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0,10348/1,10348,1,14.4,15,0.15,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
4,1,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0,10354/1,10354,1,14.4,12,0.00,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"


Viewing the head of the merged table we can see lots of variables we don't need. We will create a new DataFrame with only the categories we need: 'Quantity', 'Discount', and 'CategoryName'.

In [12]:
q2_data = prod_cat[['Quantity', 'Discount', 'CategoryName']]
q2_data.head()

,Quantity,Discount,CategoryName
0,45,0.20,Beverages
1,18,0.00,Beverages
2,20,0.00,Beverages
3,15,0.15,Beverages
4,12,0.00,Beverages


Since we know from before that there are a few 'Discount' values with signficantly less occurances than others, we will need to address that for this question. Let's view the value counts for 'Discount'.

In [13]:
q2_data['Discount'].value_counts()

0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Discount, dtype: int64

Since the low occurance rate 'Discount' values (0.01, 0.02, 0.03, 0.04, and 0.06) are all close to 0.05. We will simply adjust them each to 0.05 to give us a more equal distribution of discounts.

In [14]:
q2_data['Discount'] = q2_data['Discount'].apply(lambda x: 0.05 if (x > 0 and x < 0.10) else x)

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Since we want to test the impact of a variable ('Discount') on the dependent variable ('Quantity') for multiple groups ('CategoryName'), we will use analysis of variance or ANOVA to test our hypothesis. 

In [17]:
formula = 'Quantity ~ C(CategoryName)*C(Discount)'
lm = ols(formula, q2_data).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

,sum_sq,df,F,PR(>F)
C(CategoryName),1148.391892,7.0,0.466377,8.593555e-01
C(Discount),15825.229259,5.0,8.997563,1.794405e-08
C(CategoryName):C(Discount),21108.931647,35.0,1.714522,5.838482e-03
Residual,741173.098204,2107.0,NaN,NaN


From the above results we can see that the p-value for the interaction between 'Discount' and 'CategoryName' is approx. 0.0058. Since this is less than our alpha of 0.05, we can reject the null hypothesis and accept the alternative hypothesis. This means that we can say there is evidence that the interaction of 'Discount' and 'CategoryName' has a statistically significant effect on the quantity of product ordered.

We can do some additional analysis to look at each category individually to see if maybe certain categories show more of an effect on quantity when a discount is given. By using ANOVA again but this time having only one independent variable, 'Discount_bin', we can see the individual effect on 'Quantity' for each category. We will store the results in an array called 'results' with the 'CategoryName', F-value, and p-value in that order.

In [35]:
results = []
for i in q2_data['CategoryName'].unique():
    i_data = q2_data[q2_data['CategoryName'] == i]
    for j in i_data['Discount'].unique():
        j_data = i_data[i_data['Discount'] == j]
        formula = 'Quantity ~ '
        lm = ols(formula, j_data).fit()
        table = sm.stats.anova_lm(lm, typ=1)
        results.append([str(i) + " + " + str(j), table['F'].iloc[0], table['PR(>F)'].iloc[0]])

PatsyError: expected a noun, but instead the expression ended
    Quantity ~
             ^

Since we are doing some post-hoc testing, we need to use a Bonferroni correction to avoid p-hacking. The adjusted alpha is calculated below.

In [25]:
n = len(q2_data['CategoryName'].unique())
alpha = 0.05
bon_alpha = alpha/n
bon_alpha

0.00625

With our new adjusted alpha, we can now view the results where the p-value is less than our 'bon_alpha' value.

In [32]:
for i in results:
    if i[2] < bon_alpha:
        print(i)

We can see that 3 of the categories produced results with significant probabilities. Out of these, 'Beverages' and 'Dairy Products' both have high F-values of ~13.74 and ~13.56 respectively, while 'Condiments' has a lesser F-value of ~9.1. From these results, it would suggest that for products in the categories of 'Beverages', 'Dairy Products', and 'Condiments', providing a discount will almost always increase the quantity of product sold. Furthermore, that increase in product sold will be highest in 'Beverages' and 'Dairy Products'.

### Our findings show that having a discount on a product most likely will increase the quantity of product order but the actual amount of discount is statistically insignificant. 

In [21]:
formula = 'Quantity ~ C(CategoryName)*C(Discount_bin)'
lm = ols(formula, q2_data).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

PatsyError: Error evaluating factor: NameError: name 'Discount_bin' is not defined
    Quantity ~ C(CategoryName)*C(Discount_bin)
                               ^^^^^^^^^^^^^^^

First let's do what we did in the first question and place the 'Discount' values into buckets.

In [ ]:
q2_data['Discount'].value_counts()

In [ ]:
q2_data['Discount'] = q2_data['Discount'].apply(lambda x: 0.05 if (x > 0 and x < 0.10) else x)
q2_data['Discount'].value_counts()

In [ ]:
formula = 'Quantity ~ C(CategoryName)*C(Discount)'
lm = ols(formula, q2_data).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

In [ ]:
from statsmodels.graphics.factorplots import interaction_plot
fig = interaction_plot(q2_data.Discount, q2_data.CategoryName, q2_data.Quantity,
             colors=['red','blue','green','yellow','purple','orange','pink', 'black'], ms=10)

In [ ]:
formula = 'Quantity ~ C(Discount)*C(CategoryName)'
lm = ols(formula, q2_data).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

In [ ]:
formula = 'Quantity ~ C(CategoryName) + C(Discount)'
lm = ols(formula, q2_data).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

In [ ]:
q2_data = q2_data.join(pd.get_dummies(q2_data['CategoryName']))
q2_data.columns = [c.replace(' ', '_') for c in q2_data.columns]
q2_data.columns = [c.replace('/', '_') for c in q2_data.columns]
q2_data.head()

In [ ]:
for i in q2_data['CategoryName'].unique():
    data = q2_data[q2_data['CategoryName'] == i]
    formula = 'Quantity ~ C(Disc)'
    lm = ols(formula, data).fit()
    table = sm.stats.anova_lm(lm, typ=2)
    print(i)
    print(table)

In [ ]:
X = order_details['Quantity'][(order_details['Discount'] < 0.15) & (order_details['Discount'] > 0)]
y = order_details['Quantity'][(order_details['Discount'] > 0.15)]
p = stats.ttest_ind(X, y)[1]
p

In [ ]:
orders.ShipCountry.value_counts()

In [ ]:
orders.head()

In [ ]:
order_countries = orders[['Id','ShipCountry']]

In [ ]:
order_disc_quan = order_details[['OrderId','Discount','Quantity']]

In [ ]:
df = pd.merge(order_countries, 
                  order_disc_quan,
                  left_on='Id',
                  right_on='OrderId',
                  how='left')

In [ ]:
df.head()

In [ ]:
df = df.join(pd.get_dummies(df.ShipCountry))


In [ ]:
df.head()

In [ ]:
columns = "+Discount*".join(df.drop(['Quantity', 'OrderId', 'Id', 'Discount'], axis=1).columns)
f = 'Quantity~' + columns
round(ols(formula=f, data=df).fit().pvalues,5)

In [ ]:
order_details.Discount.hist()

In [ ]:
disc_05 = order_details[['Quantity','Discount']][(order_details.Discount<0.05) & (order_details.Discount>0)]
disc_15 = order_details[['Quantity','Discount']][(order_details.Discount<0.15) & (order_details.Discount > 0.05)]
disc_25 = order_details[['Quantity','Discount']][order_details.Discount>0.15]
print("Less than 5% : " + str(round(ols(formula='Quantity ~ Discount', data=disc_05).fit().pvalues[1],5)))
print("Between 5% and 15% : " + str(round(ols(formula='Quantity ~ Discount', data=disc_15).fit().pvalues[1],5)))
print("Between 15% and 25% : " + str(round(ols(formula='Quantity ~ Discount', data=disc_25).fit().pvalues[1],5)))

In [ ]:
d = 0
for i in sorted(order_details.Discount.unique()):
    d = order_details[['Quantity', 'Discount']][order_details.Discount==i].copy()
    print(str(i)+ " : " + str(ols(formula='Quantity ~ Discount', data=d).fit().pvalues[1]))

In [ ]:
disc_5 = order_details['Quantity'][order_details['Discount'] <= 0.05]
disc_10 = order_details['Quantity'][(order_details['Discount'] > 0.05) & (order_details['Discount'] <= 0.1)]
disc_15 = order_details['Quantity'][(order_details['Discount'] > 0.1) & (order_details['Discount'] <= 0.15)]
disc_20 = order_details['Quantity'][(order_details['Discount'] > 0.15) & (order_details['Discount'] <= 0.2)]
disc_25 = order_details['Quantity'][order_details['Discount'] > 0.2]
buckets = [(0.05, disc_5), (0.1, disc_10), (0.15, disc_15), (0.2, disc_20), (0.25,disc_25)]
c = 0 
while c < len(buckets) - 1:
    x = buckets[c][1]
    y = buckets[c+1][1]
    p = stats.ttest_ind(x,y)[1]
    print(str(buckets[c][0]) + " v.s. " + str(buckets[c+1][0]) + " : " + str(p)) 
    c += 1

In [ ]:
for i in sorted(disc.Discount.unique(), reverse=True):
    x = order_details['Quantity'][order_details['Discount'] <= i]
    y = no_disc.Quantity
    p = stats.ttest_ind(x,y)[1]
    print(str(i) + " or less : " + str(p))

In [ ]:
order_details.Discount.value_counts()

In [ ]:
for i in sorted(disc.Discount.unique()):
    x = order_details['Quantity'][order_details['Discount'] >= i]
    y = no_disc.Quantity
    p = stats.ttest_ind(x,y)[1]
    print(str(i) + " or higher : " + str(p))

In [ ]:
prod_ord_details = pd.merge(products, 
                  order_details,
                  left_on='Id',
                  right_on='ProductId',
                  how='left')
prod_ord_details.head()

In [ ]:
prod_ord_details = pd.merge(products, 
                  order_details,
                  left_on='Id',
                  right_on='ProductId',
                  how='left')
prod_ord_details_cat = pd.merge(prod_ord_details, 
                  categories,
                  left_on='CategoryId',
                  right_on='Id',
                  how='left')
prod_ord_details_cat.head()

In [ ]:
df_cat_disc = prod_ord_details_cat.filter(['Quantity', 'Discount','CategoryName'], axis=1)
df_cat_disc = prod_ord_details_cat.join(pd.get_dummies(df_cat_disc.CategoryName))
df_cat_disc.CategoryName.unique()

In [ ]:
a = prod_ord_details_cat[['Quantity', 'Discount', 'CategoryName']].copy()
a = a.join(pd.get_dummies(a.CategoryName))
a.drop('CategoryName', axis=1, inplace=True)
a.columns = [c.replace(' ', '_') for c in a.columns]
a.columns = [c.replace('/', '_') for c in a.columns]

In [ ]:
a.head()

In [ ]:
cats = a.iloc[:,3:].columns
for i in cats:
    x = a['Quantity'][(a[i]== 1) & (a['Discount'] > 0)]
    y = a['Quantity'][(a[i]== 1) & (a['Discount'] == 0)]
    p = stats.ttest_ind(x,y)[1]
    print(i + " : " + str(p))

In [ ]:
ord_det = pd.merge(order_details, 
                  orders,
                  left_on='OrderId',
                  right_on='Id',
                  how='left')
ord_det_emp = pd.merge(ord_det,
                      employees,
                      left_on='EmployeeId',
                      right_on='Id',
                      how='left')
ord_det_emp.info()

In [ ]:
for i in ord_det_emp.Id.unique():
    sales = (ord_det_emp['Quantity'][ord_det_emp['Id'] == i]) * (ord_det_emp['UnitPrice'][ord_det_emp['Id'] == i])
    print (str(i) + " " + ord_det_emp['Title'][ord_det_emp['Id'] == i].unique() + " : " + str(round(sales.sum(),1)))
    #print(ord_det_emp[i])

In [ ]:
def get_sales(ID):
    sales = (ord_det_emp['Quantity'][ord_det_emp['Id'] == ID]) * (ord_det_emp['UnitPrice'][ord_det_emp['Id'] == ID])
    return sales

In [ ]:
def get_total_sales(ID):
    sales = (ord_det_emp['Quantity'][ord_det_emp['Id'] == ID]) * (ord_det_emp['UnitPrice'][ord_det_emp['Id'] == ID])
    return sales.sum()

In [ ]:
x = []
for i in [6,4,3,9,1,7]:
    x.append(get_total_sales(i))
y = []
for i in [5,8,2]:
    y.append(get_total_sales(i))
stats.ttest_ind(x,y)

In [ ]:
for i in sorted(ord_det_emp.Id.unique()):
    for j in sorted(ord_det_emp.Id.unique()):
        if j <= i:
            pass
        else:
            x = get_sales(i)
            y = get_sales(j)
            p = stats.ttest_ind(x,y)[1]
            if p <= 0.05:
                print(str(i) + " v.s. " + str(j) + " : " + str(round(p,3)))
            

In [ ]:
ord_det_emp.BirthDate.unique()

In [ ]:
ord_det_emp.Id.unique()

In [ ]:
reps_data = []
for i in sales_reps_IDs:
    ord = orders['Id'][orders['EmployeeId'] == i]
    details = []
    for j in ord:
        details.append(order_details[order_details['OrderId'] == j])
    reps_data.append([i, details])


In [ ]:
for i in range (0, len(reps_data)):
    print(pd.DataFrame(reps_data[i][1][1].info()))

In [ ]:
orders.EmployeeId.value_counts()